In [2]:
from fastNLP import DataSet

data_path = './data/sample_dataset.csv'
ds = DataSet.read_csv(data_path, headers=('raw_sentence','label'), sep='\t')

In [3]:
ds[1]

{'raw_sentence': This quiet , introspective and entertaining independent is worth seeking . type=str,
'label': 4 type=str}

In [8]:
ds.apply(lambda x: x['raw_sentence'].lower(), new_field_name='raw_sentence')

ds.apply(lambda x: int(x['label']), new_field_name='label_seq',is_target=True)

def split_sent(ins):
    return ins['raw_sentence'].split()
ds.apply(split_sent, new_field_name='words',is_input=True)
ds

DataSet({'raw_sentence': a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story . type=str,
'label': 1 type=str,
'label_seq': 1 type=int,
'words': ['a', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gander', ',', 'some', 'of', 'which', 'occasionally', 'amuses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.'] type=list},
{'raw_sentence': this quiet , introspective and entertaining independent is worth seeking . type=str,
'label': 4 type=str,
'label_seq': 4 type=int,
'words': ['this', 'quiet', ',', 'introspective', 'and', 'entertaining', 'independent', 'is', 'worth', 'seeking', '.'] type=list},
{'raw_sentence': even fans of ismail merchant 's work , i suspect , would have a hard time sitting through this one . type=str,
'l

In [9]:
# 分割训练集/验证集
train_data, dev_data = ds.split(0.3)
print('Train size:',len(train_data))
print('Test size:',len(dev_data))

Train size: 54
Test size: 23


In [11]:
from fastNLP import Vocabulary
vocab =  Vocabulary(min_freq=2)
train_data.apply(lambda x:[vocab.add(word) for word in x['words']])

train_data.apply(lambda x:[vocab.to_index(word) for word in x['words']], new_field_name='word_seq',is_input=True)
dev_data.apply(lambda x: [vocab.to_index(word) for word in x['words']], new_field_name='word_seq', is_input=True)

In [16]:
from fastNLP.models import CNNText
model = CNNText(embed_num=len(vocab), embed_dim=50, num_classes=5, padding=2, dropout=0.1)

In [21]:
from fastNLP import Trainer, CrossEntropyLoss, AccuracyMetric
trainer = Trainer(model=model,
                  train_data=train_data,
                  dev_data=dev_data,
                  loss=CrossEntropyLoss(),
                  metrics=AccuracyMetric(),
#                   epoch=10
                )
trainer.train()
print('Train finished!')    

input fields after batch(if batch size is 2):
	words: (1)type:numpy.ndarray (2)dtype:object, (3)shape:(2,) 
	word_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2, 37]) 
target fields after batch(if batch size is 2):
	label_seq: (1)type:torch.Tensor (2)dtype:torch.int64, (3)shape:torch.Size([2]) 

training epochs started 2019-04-02-20-28-18


Evaluation at Epoch 1/3. Step:2/6. AccuracyMetric: acc=0.695652
Evaluation at Epoch 2/3. Step:4/6. AccuracyMetric: acc=0.782609
Evaluation at Epoch 3/3. Step:6/6. AccuracyMetric: acc=0.73913

In Epoch:2/Step:4, got best dev performance:AccuracyMetric: acc=0.782609
Reloaded the best model.
Train finished!
